In [1]:
import sys
import subprocess
import pkg_resources
import pandas as pd
import numpy as np
import sklearn
import sqlalchemy as sa
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import calendar
from datetime import datetime
from dateutil.relativedelta import relativedelta
import joblib
from joblib import dump, load

# one-hot encoding
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder

# model(s)
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# train test split
from sklearn.model_selection import train_test_split

# class imbalance
from collections import Counter
from sklearn.utils.class_weight import compute_sample_weight

# eval metrics
from sklearn.metrics import precision_score, recall_score, precision_recall_curve, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from math import sqrt

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

import os
os.getcwd()
print(os.environ.get('CONDA_DEFAULT_ENV'))
assert os.environ.get('CONDA_DEFAULT_ENV') == 'sagemaker_renforecast_env', 'Error! Wrong environment will cause downstream versioning issues.'

installed_packages = [f"{dist.key} {dist.version}" for dist in pkg_resources.working_set]
for package in installed_packages:
    print(package)

# Install specific sklearn version (for Sagemaker)
subprocess.check_call([sys.executable, "-m", "pip", "install", "scikit-learn==1.2.1"])

# Verify installation
print(f"Scikit-learn version: {sklearn.__version__}")

C:\Users\ybai\AppData\Local\Temp\ipykernel_19804\3978857099.py:3: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


sagemaker_renforecast_env
pyyaml 6.0.2
annotated-types 0.7.0
antlr4-python3-runtime 4.9.3
anyio 4.9.0
asttokens 3.0.0
attrs 23.2.0
boto3 1.38.3
botocore 1.38.3
category-encoders 2.6.0
certifi 2025.4.26
charset-normalizer 3.4.1
click 8.1.8
cloudpickle 3.1.1
colorama 0.4.6
comm 0.2.2
contourpy 1.3.2
cycler 0.12.1
debugpy 1.8.14
decorator 5.2.1
dill 0.4.0
docker 7.1.0
exceptiongroup 1.2.2
executing 2.2.0
fastapi 0.115.12
fonttools 4.57.0
google-pasta 0.2.0
greenlet 3.2.2
h11 0.16.0
idna 3.10
importlib-metadata 6.11.0
ipykernel 6.29.5
ipython 8.36.0
jedi 0.19.2
jmespath 1.0.1
joblib 1.4.2
jsonschema 4.23.0
jsonschema-specifications 2025.4.1
jupyter-client 8.6.3
jupyter-core 5.7.2
kiwisolver 1.4.8
markdown-it-py 3.0.0
matplotlib 3.10.1
matplotlib-inline 0.1.7
mdurl 0.1.2
mkl-fft 1.3.11
mkl-random 1.2.8
mkl-service 2.4.0
mock 4.0.3
multiprocess 0.70.18
nest-asyncio 1.6.0
numpy 1.26.4
omegaconf 2.3.0
packaging 24.2
pandas 2.2.3
parso 0.8.4
pathos 0.3.4
patsy 1.0.1
pillow 11.2.1
pip 25.0
platf

In [ ]:
# set connection parameters
driver = "ODBC Driver 17 for SQL Server"
server = "SQLWRH22"
database = "NimbleReplica"
username = "ybai"

### SQLALCHEMY METHOD ###
# define connection
connection_string = "DRIVER={%s}; \
                     SERVER=%s; \
                     DATABASE=%s;\
                     UID=%s; \
                     Trusted_Connection=yes" % (driver, server, database, username)
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)

query = """
SET NOCOUNT ON
--THIS IS THE QUERY FOR MBR RENEWAL FORECAST, ADAPTED FROM THE MMQ
;with cteNextTerm(NU__Account2__c,ControlGroupID__c, ControlGroupDate__c, CreatedDate, NU__ExternalID__c, Next_Term, 
                  Next_SubRef, Next_startdate, Next_enddate, Id, NU__OrderItemLine__c, NU__MembershipType2__c, NU__Status__c,KeyCode__c, NU__AutoRenew__c, CTC__c)
AS
(
  select NU__Account2__c, ControlGroupID__c, ControlGroupDate__c, CreatedDate, NU__ExternalID__c, Right(NU__ExternalID__c,(Len(NU__ExternalID__c) - 13)) as 'Next_Term', 
        Left(NU__ExternalID__c, 12) as 'Next_SubRef', NU__StartDate__c 'Next_startdate', NU__EndDate__c 'Next_enddate', Id, NU__OrderItemLine__c, 
        NU__MembershipType2__c, NU__Status__c, KeyCode__c, NU__AutoRenew__c, CTC__c
    from NU__Membership__c with(NOLOCK)
    where
        ISNUMERIC(Left(NU__ExternalID__c, 12)) = 1
        AND ISNUMERIC(Right(NU__ExternalID__c, Len(NU__ExternalID__c) - 13)) = 1
),
     
	 ctePrvTerm(NU__Account2__c,ControlGroupID__c, ControlGroupDate__c, CreatedDate, NU__ExternalID__c, Prv_Term, 
        Prv_SubRef, Prv_startdate, Prv_enddate, Id,NU__Status__c,KeyCode__c, NU__AutoRenew__c, CTC__c)   
   AS
   (
    select NU__Account2__c, ControlGroupID__c, ControlGroupDate__c, CreatedDate, NU__ExternalID__c, Right(NU__ExternalID__c,(Len(NU__ExternalID__c) - 13)) as 'Prv_Term', 
        Left(NU__ExternalID__c, 12) as 'Prv_SubRef' , NU__StartDate__c 'Prv_startdate', NU__EndDate__c 'Prv_enddate', Id,NU__Status__c,KeyCode__c, NU__AutoRenew__c, CTC__c
    from NU__Membership__c with(NOLOCK)
    where
        ISNUMERIC(Left(NU__ExternalID__c, 12)) = 1
        AND ISNUMERIC(Right(NU__ExternalID__c, Len(NU__ExternalID__c) - 13)) = 1
)

select 
case when Acc.CustomerNumber__c is null then ltrim(rtrim(right(left(Acc.NU__ExternalID__c,12),8))) else Acc.CustomerNumber__c end as CustomerNumber__c,
mem.NU__AutoRenew__c,
Acc.PersonEmail 'Email',
Memtyp.Name 'Member_Type', 
mem.KeyCode__c,
Case
		when Mem.ControlGroupID__c <> '' and Mem.ControlGroupDate__c < mem.CreatedDate
			then cast(Mem.ControlGroupDate__c as date)
			else cast(mem.CreatedDate as date) end as 'OrderDate', --Order_Created_Date
cast(PrvMem.Prv_enddate as date) 'Prev_Term_End_Date', -- Getting the Pervious Term End Date
cast(Mem.NU__StartDate__c as date) 'CurrentMem_Start_Date', -- Getting the Current Membership Term Start Date
cast(Mem.NU__EndDate__c as date) 'CurrentMem_End_Date', -- Getting the Current Membership Term End Date
cast(NxMem.Next_startdate as date) 'NextMem_Start_Date',-- Getting the Next Membership Term Start Date
cast(NxMem.Next_enddate as date) 'NextMem_End_Date',
datediff(day, mem.NU__StartDate__c, mem.NU__EndDate__c) 'Tenure', --Tenure in days
Right(Mem.NU__ExternalID__c,(Len(Mem.NU__ExternalID__c) - 13)) as 'Adv_Term',
mem.Id,
Case 
		When Len(O.NU__ExternalId__c) = 8 AND Left(O.NU__ExternalId__c,1) IN ('8','B') then OIL.NU__UnitPrice__c
		when O.NU__SubTotal__c IS NOT NULL then O.NU__SubTotal__c
        else Mem.Rate__c
        end as 'Membership_Rate',
case
		When Mem.NU__Status__c = 'Grace' then 'Gracing'
		When Mem.NU__Status__c = 'Current' and Mem.NU__EndDate__c < getdate() and NxMem.Next_startdate is null then 'Gracing'
	else Mem.NU__Status__c end as 'Membership_Status',
prod.Name as 'Product_Type',

--update Renewed YB 0529
--Case when NxMem.Next_startdate is not null then 1 end as 'Renewed',
Case
	--When there is a renewal count it
	--Updated to also look at status since future term could be canceled which would mean the member is not renewed
	when NxMem.Next_startdate is not null and NxMem.NU__Status__c <> 'Cancelled' then 1
	else 0
	end as 'Renewed',

--add Membership Audience and election year and post-election year. YB June 2025
case
		When Memtyp.Name IN ('Lifetime', 'Emeritus', 'Emeritus without Science', 'Science Advocate','Supporting','Supporting without Science','Retired') then 'Enthusiast'
		When Memtyp.Name IN ('Silver','Gold','Platinum','Regular','Post Doc','NPA Post Doc','Patron','Paid Life','Teacher','Teacher with SBF') then 'Scientist'
		When Memtyp.Name IN ('Student','Student Gold','Student Silver') then 'Student'
		When Memtyp.Name IN ('Sponsored','Sponsored Student Silver') then 'Sponsored'
		When Memtyp.Name IN ('Elemental') then 'Elemental'
	else Memtyp.Name end as 'Membership_Audience',

case 
	when year(Mem.NU__EndDate__c) = 2024 then 1
	else 0
	end as 'Election_Year',

case
	when year(mem.NU__EndDate__c) = 2025 then 1
	else 0
	end as 'Post_Election_Year',


case
 	when acc.BirthYear__pc in ('Prefer not to Answer') then NULL  
	when acc.BirthYear__pc is null then NULL
	when cast(year(getdate())- acc.BirthYear__pc as varchar) between 0 and 15 or cast(year(getdate())- acc.BirthYear__pc as varchar) >= 101 then NULL
	else cast(year(getdate())- acc.BirthYear__pc as varchar)
	end as 'Age',

SIN(2 * PI() * (MONTH(Mem.NU__EndDate__c) - 1) / 12) AS Month_sin,

COS(2 * PI() * (MONTH(Mem.NU__EndDate__c) - 1) / 12) as Month_cos
	
from NU__Membership__c mem
left join cteNextTerm NxMem 
	on  Left(Mem.NU__ExternalID__c,12) = NxMem.Next_SubRef
    and Right(Mem.NU__ExternalID__c,(Len(Mem.NU__ExternalID__c) - 13)) = NxMem.Next_Term -1

left join ctePrvTerm  PrvMem 
	on  Left(Mem.NU__ExternalID__c,12) = PrvMem.Prv_SubRef
    and Right(Mem.NU__ExternalID__c,(Len(Mem.NU__ExternalID__c) - 13)) = PrvMem.Prv_Term + 1

left join Account acc on mem.NU__Account2__c = acc.Id
left join NU__MembershipType__c MemTyp on Mem.NU__MembershipType2__c = Memtyp.ID
left join NU__Subscription__c Sub on mem.NU__ExternalID__c = sub.NU__ExternalID__c
left join NU__Product__c prod on sub.NU__Product2__c = prod.id
left join NU__OrderItemLine__c OIL on mem.NU__OrderItemLine__c = OIL.ID
left join NU__OrderItem__c OI on OIL.NU__OrderItem__c = OI.ID
left join NU__Order__C O on OI.NU__Order__c = O.ID

where 
Acc.PersonEmail not like 'qauser%' and
Acc.PersonEmail not like '%@aaas.org' and 
Acc.PersonEmail not like '%@example.com' and
Year(Mem.NU__EndDate__c) >= (year(GETDATE()) - 4) and Year(Mem.NU__EndDate__c) < (Year(getdate())+2) --June 2025 YB: remove 2020 data as it does not fit the pattern
    and LEN(Mem.NU__ExternalId__c) > 13
    and (LEN(NxMem.NU__ExternalId__c) > 13 or NxMem.NU__ExternalID__c is null)
and(
ISNUMERIC(Left(Mem.NU__ExternalID__c, 12)) = 1
AND ISNUMERIC(Right(Mem.NU__ExternalID__c, Len(Mem.NU__ExternalID__c) - 13)) = 1
)
	and NOT (Memtyp.Name = 'Supporting' AND Year(Mem.NU__EndDate__c) < 2020)
;


"""

# retrieve data using engine connection
with engine.begin() as conn:
    renewal_df = pd.read_sql_query(sa.text(query), conn)

In [ ]:
data = renewal_df

# Do NOT remove any of the fields below. You need it for data quality checks.
list_of_features = ['CustomerNumber__c',
    'NU__AutoRenew__c',
    'Email',
    'Member_Type',
    'KeyCode__c',
    'CurrentMem_Start_Date',
    'CurrentMem_End_Date',
    'OrderDate',
    'Prev_Term_End_Date',
    'NextMem_Start_Date',
    'NextMem_End_Date',
    'Tenure',
    'Adv_Term',
    'Membership_Status',
    'Product_Type',
    'Membership_Rate',
    'Renewed',
    'Membership_Audience',
    'Election_Year',
    'Post_Election_Year',
    'Age',
    'Month_sin',
    'Month_cos'
    ]
data = data[list_of_features]

# Code Renewal Date
sorted_data = data.sort_values(['Email', 'CurrentMem_Start_Date'], ascending=[True, True])
conditions = [sorted_data['Email'] == sorted_data['Email'].shift(-1)]
choices = [sorted_data['OrderDate'].shift(-1)]
sorted_data['RenewalDate'] = np.select(conditions, choices, default=None)
data = sorted_data

# print dataframe to observe results
data_check_renewaldate = data[['Email','CurrentMem_Start_Date','CurrentMem_End_Date','OrderDate','RenewalDate','Adv_Term']]
display(data_check_renewaldate)

# Code if there is a next term
conditions_next_term = [data.sort_values(['Email', 'CurrentMem_Start_Date'], ascending=[True, True])['Email'] == data.sort_values(['Email', 'CurrentMem_Start_Date'], ascending=[True, True])['Email'].shift(-1)]
choices_next_term = ["Next_Term"]
data['Next_Term'] = np.select(conditions_next_term, choices_next_term, default=None)

data_check_nextterm = data[['Email','CurrentMem_Start_Date','CurrentMem_End_Date','OrderDate','RenewalDate','Adv_Term','Next_Term']]
data_check_nextterm

data = data.reset_index(drop=True)

# Convert Data Types
conversion_dict = {
        'OrderDate':'datetime64[ns]',
        'Prev_Term_End_Date': 'datetime64[ns]',
        'CurrentMem_Start_Date': 'datetime64[ns]',
        'CurrentMem_End_Date': 'datetime64[ns]',
        'NextMem_Start_Date': 'datetime64[ns]',
        'NextMem_End_Date': 'datetime64[ns]',
        'RenewalDate': 'datetime64[ns]',
        'Adv_Term': 'Int32',
        'Election_Year': 'bool',
        'Post_Election_Year': 'bool',
        'Age': 'Int64',
        'Renewed': 'Int32'
    }

data = data.astype(conversion_dict)
print(data.dtypes)

# RenewalCurve_type
# Filter membership rates and fill Product Type blanks
RenewalCurve_type = data
RenewalCurve_type.shape

RenewalCurve_type['Membership_Rate'].notna().value_counts() # count NA membership rates
RenewalCurve_type[RenewalCurve_type['Membership_Rate'] < 0].value_counts()
RenewalCurve_type = RenewalCurve_type[(RenewalCurve_type['Membership_Rate'].notna()) & (RenewalCurve_type['Membership_Rate'] >= 0)]

RenewalCurve_type['Product_Type'] = RenewalCurve_type['Product_Type'].fillna('No Science')

# Keep copy of dataset
RenewalCurve_type_copy = RenewalCurve_type # keep a copy of the dataset

# filter prev term end date = null but adv_term is not 1
mask_advterm_enddate = (RenewalCurve_type['Prev_Term_End_Date'].isna()) & (RenewalCurve_type['Adv_Term'] != 1) # Adv_Term must be an int for this to work
data_mask_advterm_enddate = RenewalCurve_type[mask_advterm_enddate]
RenewalCurve_type = RenewalCurve_type[~RenewalCurve_type['CustomerNumber__c'].isin(data_mask_advterm_enddate['CustomerNumber__c'])]

print(RenewalCurve_type.shape)

# Filter skipping Adv_Terms
RenewalCurve_type = RenewalCurve_type.sort_values(['CustomerNumber__c','Adv_Term'], ascending=[True, True]).reset_index(drop=True)
RenewalCurve_type_test = RenewalCurve_type
# filter skipping terms
conditions_check2 = [RenewalCurve_type.shift(-1)['CustomerNumber__c'] == RenewalCurve_type['CustomerNumber__c']]
choices_check2 = [RenewalCurve_type.shift(-1)['Adv_Term'] - RenewalCurve_type['Adv_Term']]
RenewalCurve_type_test['Term_Diff'] = np.select(conditions_check2, choices_check2, default=0)
# code term diff as int
RenewalCurve_type_test['Term_Diff'] = RenewalCurve_type_test['Term_Diff'].astype(int)
# create df where Term_Diff is larger than 1 (invalid for the same customer)
RenewalCurve_type_test = RenewalCurve_type_test[RenewalCurve_type_test['Term_Diff'] > 1].sort_values(by=['CustomerNumber__c','Term_Diff'], ascending=[True,False])
# remove duplicates in RenewalCurve_type_test
RenewalCurve_type_test = RenewalCurve_type_test.drop_duplicates(subset=['CustomerNumber__c'], keep='first')
# filter out records with duplicates
RenewalCurve_type = RenewalCurve_type[~(RenewalCurve_type['CustomerNumber__c'].isin(RenewalCurve_type_test['CustomerNumber__c']))]
# confirm all members with skipping Adv_Terms have been removed
c_with_large_diff = RenewalCurve_type.groupby('Email')['Term_Diff'].transform('max') > 1
filtered_df = RenewalCurve_type[c_with_large_diff]
filtered_df = filtered_df.sort_values(by=['Email', 'Term_Diff'], ascending=[True, False])
assert filtered_df.shape == (0,len(RenewalCurve_type.columns)), 'Error! Skipping Adv_Terms not removed'

# Drop members with membership dates that don't make sense
non_dates = RenewalCurve_type
mask_non_dates = ( (non_dates['Prev_Term_End_Date'] > non_dates['CurrentMem_Start_Date']) |
                 (non_dates['CurrentMem_Start_Date'] > non_dates['CurrentMem_End_Date']) |
                 (non_dates['CurrentMem_End_Date'] > non_dates['NextMem_Start_Date']) |
                 (non_dates['NextMem_Start_Date'] > non_dates['NextMem_End_Date']) )
non_dates = non_dates[mask_non_dates]

RenewalCurve_type = RenewalCurve_type[~RenewalCurve_type['CustomerNumber__c'].isin(non_dates['CustomerNumber__c'])]
RenewalCurve_type.info()


# Define function to process dataset for specific membership types
def filter_member_type(dataframe):
    mask_final_filter = (dataframe['Member_Type'].isin(['Gold','Platinum','Silver',
                                                                   'Regular','Student Silver','Student Gold',
                                                                   'Student','Science Advocate','Retired','Teacher',
                                                                   'Post Doc','Emeritus','Supporting'])) 
    
    data = dataframe[mask_final_filter]

    return data

RenewalCurve_type = filter_member_type(RenewalCurve_type)

## Feature Eng

In [ ]:
# 250605 YB: Shuold modularize this with code that cleans data and adds features (respetcively). Deal with this later.
RenewalCurve_type['Adv_Term'] = RenewalCurve_type['Adv_Term'].astype(int) # make sure adv_term is an integer

### difference resolved: R code did not convert Adv_Term into integer before creating bins
# create TermGroupChanged
conditions_termgroupchanged = [(RenewalCurve_type['Adv_Term'] == 1), 
                        (RenewalCurve_type['Adv_Term'] == 2),
                        (RenewalCurve_type['Adv_Term'] == 3), 
                        (RenewalCurve_type['Adv_Term'] >= 4)]
choices_termgroupchanged = ['Term 1', 'Term 2', 'Term 3', 'Term 4+']
RenewalCurve_type['TermGroupChanged'] = np.select(conditions_termgroupchanged, choices_termgroupchanged, default=pd.NA)
RenewalCurve_type.value_counts('TermGroupChanged')

# create Term_Group
conditions_termgroup = [(RenewalCurve_type['Adv_Term'] == 1), 
                        ((RenewalCurve_type['Adv_Term'] == 2) | (RenewalCurve_type['Adv_Term'] == 3)),
                        (RenewalCurve_type['Adv_Term'] >= 4)]
choices_termgroup = ['Term 1', 'Term 2-3', 'Term 4+']
RenewalCurve_type['Term_Group'] = np.select(conditions_termgroup, choices_termgroup, default=pd.NA)
RenewalCurve_type.value_counts('Term_Group')

## 0602 YB add month and year to the dataset for monthly budgets
RenewalCurve_type['CurrentMem_End_Month'] = RenewalCurve_type['CurrentMem_End_Date'].dt.month.astype(int)
RenewalCurve_type['CurrentMem_End_Month_Name'] = RenewalCurve_type['CurrentMem_End_Date'].dt.month_name().astype(str)
RenewalCurve_type['CurrentMem_End_Year_Int'] = RenewalCurve_type['CurrentMem_End_Date'].dt.year # for filtering based on year
RenewalCurve_type['CurrentMem_End_Year'] = RenewalCurve_type['CurrentMem_End_Date'].dt.year.astype(str)

# convert Membership Rates to log
RenewalCurve_type['Membership_Rate'] = np.log1p(RenewalCurve_type['Membership_Rate'])

RenewalCurve_type.info()

# Define function to process data and add features for both regular members and all other members
# 250605 YB: Add Month as CYCLICAL feature? did this in SQL.
def process_data_add_feature(df):
    """
    Process the input DataFrame by adding various features and converting data types.

    Args:
        df (pandas.DataFrame): The input DataFrame containing the data.

    Returns:
        df
    """
    df = df.copy()
    
# Product Type
    assert df['Product_Type'].isnull().sum() == 0, "There are still null values in the 'Product_Type' column."

# Tenure
    conditions_tenure = [
    (df['Tenure'] < 364),
    (df['Tenure'] == 364),
    (df['Tenure'] == 365),
    ((df['Tenure'] > 365) & (df['Tenure'] < 729)),
    (df['Tenure'] == 729),
    (df['Tenure'] == 730),
    (df['Tenure'] > 730)
    ]
    choices_tenure = [
        'short',
        '364_days',
        '365_days',
        'mid',
        '729_days',
        '730_days',
        'long'
    ]
    df.loc[:, 'Tenure_bins'] = np.select(conditions_tenure, choices_tenure, default=None)

    df = df.reset_index(drop=True)
    
    return df

## Final dataset(s)

In [ ]:
RenewalCurve_type2 = RenewalCurve_type.reset_index(drop=True)
mask = RenewalCurve_type2['CurrentMem_End_Date'] <= '2025-03-25'
MBR_budget_pre_2025 = RenewalCurve_type2[mask].reset_index(drop=True)
MBR_budget_2025_2026 = RenewalCurve_type2[~mask].reset_index(drop=True)

# Final data processing for member types
# training data
MBR_budget_pre_2025_processed = filter_member_type(MBR_budget_pre_2025)
# inference data
MBR_budget_2025_2026_processed = filter_member_type(MBR_budget_2025_2026)


# Automate checks for training data
MBR_budget_pre_2025_processed = MBR_budget_pre_2025_processed.reset_index(drop=True)

expected_mem_type = ['Student Gold', 'Emeritus', 'Regular', 'Student', 'Retired', 'Silver', 'Gold',
 'Science Advocate', 'Post Doc', 'Platinum', 'Supporting', 'Teacher', 'Student Silver']
assert all(mem_type in expected_mem_type for mem_type in MBR_budget_2025_2026_processed['Member_Type'].unique()), "Error! Check Membership Types."
assert all(mem_type in expected_mem_type for mem_type in MBR_budget_2025_2026_processed['Member_Type'].unique()), "Error! Check Membership Types."

# length should be the same because filter_member_type shouldn't be filtering anything out at this stage (Membership Types should already have been cleaned up)
assert len(MBR_budget_pre_2025) == len(MBR_budget_pre_2025_processed)
assert len(MBR_budget_2025_2026) == len(MBR_budget_2025_2026_processed)

In [ ]:
# Add features to training data
MBR_budget_pre_2025_processed_modeling = process_data_add_feature(MBR_budget_pre_2025_processed)
MBR_budget_pre_2025_processed_modeling_final = MBR_budget_pre_2025_processed_modeling.copy()

# Removed Member_Type, Tenure, CurrentMem_End_Year_Int due to high correlation with other features
# Need to drop more dummies after OHE
MBR_budget_pre_2025_processed_modeling_final = MBR_budget_pre_2025_processed_modeling_final[[
       'NU__AutoRenew__c', 
       'KeyCode__c',
       'Adv_Term', 
       'Product_Type', 
       'Membership_Rate',
       'Renewed', # outcome variable
       'TermGroupChanged', 
       'CurrentMem_End_Month', # this is an integer
       'Term_Group',
       'Tenure_bins',
       'Membership_Audience',
       'Age',
       'Month_sin',
       'Month_cos',
       'Election_Year',
       'Post_Election_Year'
       ]] 

mask = (MBR_budget_pre_2025_processed_modeling_final['Adv_Term'] == 1) & (MBR_budget_pre_2025_processed_modeling_final['NU__AutoRenew__c'] == False)
MBR_budget_pre_2025_processed_modeling_final['is_term1_no_auto'] = False
MBR_budget_pre_2025_processed_modeling_final.loc[mask, 'is_term1_no_auto'] = True

MBR_budget_pre_2025_processed_modeling_final['term_and_auto_renew'] = MBR_budget_pre_2025_processed_modeling_final['Adv_Term'] * MBR_budget_pre_2025_processed_modeling_final['NU__AutoRenew__c']

MBR_budget_pre_2025_processed_modeling_final['Age'] = MBR_budget_pre_2025_processed_modeling_final['Age'].fillna(value=999)

assert len(MBR_budget_pre_2025_processed_modeling_final) == len(MBR_budget_pre_2025_processed)
MBR_budget_pre_2025_processed_modeling_final.info(verbose=True)

### /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# Add features to inference data
MBR_budget_2025_2026_processed_inference = process_data_add_feature(MBR_budget_2025_2026_processed)
MBR_budget_2025_2026_processed_inference_final = MBR_budget_2025_2026_processed_inference.copy()

# Final inference dataset
MBR_budget_2025_2026_processed_inference_final = MBR_budget_2025_2026_processed_inference_final[[
       'NU__AutoRenew__c', 
       'KeyCode__c',
       'Adv_Term', 
       'Product_Type', 
       'Membership_Rate',
       'Renewed', # outcome variable
       'TermGroupChanged', 
       'CurrentMem_End_Month', # this is an integer
       'Term_Group',
       'Tenure_bins',
       'Membership_Audience',
       'Age',
       'Month_sin',
       'Month_cos',
       'Election_Year',
       'Post_Election_Year'
       ]]

mask = (MBR_budget_2025_2026_processed_inference_final['Adv_Term'] == 1) & (MBR_budget_2025_2026_processed_inference_final['NU__AutoRenew__c'] == False)
MBR_budget_2025_2026_processed_inference_final['is_term1_no_auto'] = False
MBR_budget_2025_2026_processed_inference_final.loc[mask, 'is_term1_no_auto'] = True

MBR_budget_2025_2026_processed_inference_final['term_and_auto_renew'] = MBR_budget_2025_2026_processed_inference_final['Adv_Term'] * MBR_budget_2025_2026_processed_inference_final['NU__AutoRenew__c']

MBR_budget_2025_2026_processed_inference_final['Age'] = MBR_budget_2025_2026_processed_inference_final['Age'].fillna(value=999)

assert len(MBR_budget_2025_2026_processed_inference_final) == len(MBR_budget_2025_2026_processed)
assert len(MBR_budget_pre_2025_processed_modeling_final.columns) == len(MBR_budget_2025_2026_processed_inference_final.columns)
MBR_budget_2025_2026_processed_inference_final.info()

## Save Data

In [ ]:
MBR_budget_pre_2025_processed_modeling_final.to_csv(r"C:\Users\ybai\OneDrive - AAAS\Desktop\Renewals Forecast\Sagemaker Renewal Modeling\renewal_forecast_sagemaker_data_SP2025\Renewal_Forecast_individual_modeling_dataset_FINAL_250625.csv", index=False)
MBR_budget_2025_2026_processed_inference_final.to_csv(r"C:\Users\ybai\OneDrive - AAAS\Desktop\Renewals Forecast\Sagemaker Renewal Modeling\renewal_forecast_sagemaker_data_SP2025\Renewal_Forecast_individual_inference_dataset_FINAL_250625.csv", index=False)